# Algebra lineal y cuántica

Deseamos introducir los siguientes conceptos.
* Estados aleatorios
* Proyectores
* Matrices de Pauli
* Matriz de densidad


# Definiciones basicas de vectores y algunas operaciones

In [ ]:
v=[1 2 3] # esto es un covector

In [ ]:
typeof(v)

In [1]:
# Este es un vector usual, dependiendo de la convencion que sigamos podemos tener
# mas o menos cuidado. Ayuda al principio ser un poco ordenado
v=[1,2,3.2+2im]

3-element Array{Complex{Float64},1}:
 1.0+0.0im
 2.0+0.0im
 3.2+2.0im

In [2]:
a=v' #Transposición conjugada, es decir la operacion daga o pasar de bra a ket.

1×3 Array{Complex{Float64},2}:
 1.0-0.0im  2.0-0.0im  3.2-2.0im

In [ ]:
# Producto punto usual, notese que toma el conjugado y luego el producto, por lo 
# que tenemos que el producto es real y positivo
dot(v,v)

In [ ]:
# Este es el producto exterior. Es el resultando de un ket por un bra 
# (que es diferente de un bra por un ket), y me da un proyector. Exploraremos
# un poco mas profundamente este concepto
v*v' 

In [ ]:
# Este parece el producto interno, pero notese que es un arreglo de 1x1 por lo 
# que resulta un poco inconveniente pues carga informacion irrelevante.
v'*v

In [ ]:
@show [1.0 2 3] #Arreglo de flotantes ;
@show [1+im 2 3] #arreglo de enteros complejos ;

# Estados aleatorios 1

Vamos a usar estados aleatorios. Estos se van a construir usando variables gaussianas. Dado que tenemos dos variables gaussianas con el mismo ancho, $x$ y $y$, una rotacion produce dos variables gaussianas con el mismo ancho. 

Imaginen las distribuciónes
\begin{align}
P(x) &\propto e^{-x^2}\\
P(y) &\propto e^{-y^2}.
\end{align}
entonces, la distribución conjunta estará dada por la multiplicacion de las distrubuciones:
\begin{align}
P(x,y) &\propto e^{-x^2}e^{-y^2}\\
&=e^{-r^2}
\end{align}
en coordenadas polares. 

Claramente el argumento se extiende para $n$ variables. Esto entonces nos permite elegir una dirección aleatoria con distribución uniforme, que será el punto crucial para elegir de forma apropiada estados aleatorios, que no son más que direcciones en el espacio de Hilbert complejo. 

In [ ]:
?randn # Esta funcion genera numeros aleatorios gaussianos

In [ ]:
# Pkg.add("Plots") # Quiza sea necesario para algunos bajar este paquete

In [ ]:
using Plots # Este paquete lo usamos para graficar en forma simple. 

In [ ]:
random_numbers=randn(100000);
Plots.histogram(random_numbers) # Usando histogram

* Ejercicio, mostrar la distribucion uniforme en la esfera de Bloch y mostrar como estan los puntos amontonados en los polos: Amaro, no he tenido tiempo de hacer graficas polares, te encargo les muestres esto por fas.

In [ ]:
v=randn(1,10)+randn(1,10)im # vector con numeros gaussianos aleatorios

In [ ]:
v=v/norm(v) # Podemos normalizar el estado dividiendolo por su norma;
@show norm(v) # Puede haber un error por la precision finita de la maquina;

* Pongamos todo en una función, frecuentemente es útil tener a la mano estados aleatorios.

In [ ]:
"""
Esta función crea un estado cuantico aleatorio. La dimension por defecto es 2. 
"""
function random_state(dim=2)
    v=randn(1,dim)+randn(1,dim)im
    v=v/norm(v)
    return v'
end

In [ ]:
?random_state # Se muestra la documentación de la función

In [ ]:
v=random_state() # Probando la función sin argumentos

In [ ]:
v=random_state(20);
v

In [ ]:
norm(v)

# Qubits y la esfera de Bloch

Un qubit es un sistema de dos niveles. Por lo mismo, vamos a poder representarlo como  
$$|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$$
donde se tiene la condición de normalización 
$$\langle\psi|\psi\rangle = |\alpha|^2 +|\beta|^2=1.$$
La base $|0\rangle$, $|1\rangle$ es conocida como la base computacional, y será importante cuando generalicemos a muchos qubits. 

Dadas esta condición de normalización, y dado que los estados son _rayos_ sobre el espacio de Hilbert, podemos, sin pérdida de generalidad, hacer el primer coeficiente real. Entonces, podemos elegir la siguiente representación:
$$|\psi\rangle = \cos \frac{\theta}{2} |0\rangle + e^{i \phi} \sin \frac{\theta}{2} |1\rangle.$$
Las coordenadas $\theta$ y $\phi$ corresponden a un punto en el espacio, si usamos coordenadas esféricas, y asumimos que $r=1$. Muchas (quizá todas las) operaciones de un qubit pueden ser descritas en forma transparente usando la esfera de Bloch. 

Una buena introduccion a la esfera de Bloch y en general a todos los temas clásicos de cómputo cuántico, es  
Nielsen, Michael A.; Chuang, Isaac L. (2000). Quantum Computation and Quantum Informatio. Cambridge, UK: Cambridge University Press.



# Coordenadas esféricas

Recordemos que la transformación a coordenadas esféricas es 
\begin{align}
x&=r\sin\theta\cos\phi\\
y&=r\sin\theta\sin\phi\\
z&=r\cos\theta
\end{align}

# Matrices de Pauli

Los operadores más importantes de un qubit son las matrices de Pauli. Estas están definidas como
$$
\sigma_x=
\begin{pmatrix}
  0 & 1 \\
  1 & 0
 \end{pmatrix},\quad
\sigma_y=
\begin{pmatrix}
  0 & -i \\
  i & 0
\end{pmatrix},\quad
\sigma_z=
\begin{pmatrix}
  1 & 0 \\
  0 & -1
\end{pmatrix}. 
$$

In [ ]:
sigma_x=[0. 1.; 1. 0.];
sigma_y=[0. -im; im 0];
sigma_z=[1. 0.;0. -1.];
sigmas=Array[sigma_x, sigma_y, sigma_z];

Las matrices de Pauli tiene la propiedad de que su cuadrado es 1 ($\sigma_i^2=1$), su traza es 0 ($tr \sigma_i=0$), son hermíticas y son unitarias. Estas propiedades las estaremos usando.

In [ ]:
for iter in eachindex(sigmas)
    sigma=sigmas[iter]
    @show iter
    @show norm(sigma*sigma-eye(2))+norm(sigma*sigma'-eye(2))
    @show trace(sigma)
    @show ishermitian(sigma)
end

# Un conjunto de formulas útiles

## Los valores esperados de los operadores de Pauli, como puntos en una esfera

Notamos que 
\begin{align}
\langle \psi| \sigma_x |\psi\rangle &=
\bigg(\cos \frac{\theta}{2} \langle0| + e^{-i \phi} \sin \frac{\theta}{2} \langle 1 |\bigg)
\bigg(\cos \frac{\theta}{2} |1\rangle + e^{i \phi} \sin \frac{\theta}{2} |0\rangle\bigg)\\
&= e^{-i \phi} \sin \frac{\theta}{2}\cos \frac{\theta}{2}+e^{i \phi} \sin \frac{\theta}{2}\cos \frac{\theta}{2}\\
&=2\sin \frac{\theta}{2}\cos \frac{\theta}{2}\frac{e^{-i \phi}+e^{i \phi}}{2}\\
&=\sin\theta\cos\phi\\
&=x.
\end{align}

Similarmente, 
$$\langle \psi| \sigma_y |\psi\rangle = 2\sin \frac{\theta}{2}\cos \frac{\theta}{2}\frac{e^{i \phi}-e^{-i \phi}}{2i}
=\sin\theta\sin\phi
=y
$$
y
$$\langle \psi| \sigma_z |\psi\rangle = \cos^2 \frac{\theta}{2}-\sin^2 \frac{\theta}{2}
=\cos\theta
=z.
$$

De esta manera, podemos interpretar el vector tridimensional 
$$\langle\psi|\vec \sigma|\psi\rangle$$
como un punto en una esfera, donde sus coordenadas representan los valores esperados de las matrices de Pauli.

## Proyectores y valores esperados

Derivaremos una formula que nos va a permitir introducir la matriz de densidad. Esta herramienta son los proyectores. Actuan de una manera identica a los proyectores en mecánica cuántica. Es decir, calculan la proyeccion de un vector sobre otro vector. El proyector del estado $|\psi\rangle = \sum_i c_i |i\rangle$ está dado por su producto exterior con el bra $\langle \psi| = \sum_i c_i^* \langle i|$:
\begin{align}
|\psi\rangle\langle \psi| &= \sum_{ij} c_i c_j^*|i\rangle\langle j|
\end{align}
Podemos expresar el valor esperado de un observable cualquiera 
$$
A= \sum_{ij}\alpha_{ij} |i\rangle\langle j|
$$

### ¿Que es un valor esperado?

El valor esperado de $A$ está dado por 
\begin{align}
\langle \psi| A |\psi\rangle &= \langle \psi|\sum_{ij}\alpha_{ij} |i\rangle\langle j|\sum_k c_k |k\rangle\\
 &= \langle \psi|\sum_{ij}\alpha_{ij} c_j|i\rangle \\
 &= \sum_k c_k^* \langle k| \sum_{ij}\alpha_{ij} c_j|i\rangle \\
 &= \sum_{ij} c_i^*\alpha_{ij} c_j.
\end{align}

Notemos que el valor esperado tambien se puede expresar como la traza del observable por el proyector correspondiente al estado:
\begin{align}
\text{tr} |\psi\rangle\langle \psi| A &= \text{tr} \sum_{ij} c_j c_i^* |j\rangle\langle i| \sum_{kl}\alpha_{kl} |k\rangle\langle l|\\
&=\text{tr} \sum_{ijl}c_j c_i^* \alpha_{il}|j\rangle\langle l|\\
&=\text{tr} \sum_{ij}c_j c_i^* \alpha_{ij}\\
\end{align}
por lo que concluimos que 
$$
\langle \psi| A |\psi\rangle = \text{tr} |\psi\rangle\langle \psi| A.
$$
Esto nos permite establecer un lenguaje paralelo para las mediciones, usando proyectores. Vamos a poder generalizar esto mas adeltante a otras matrices de rango mayor. 

In [ ]:
function projector(state)
    return state*state'
end

Los proyectores tienen dos características fundamentales:
* Son idempotentes ($P^2=P$)
* Tienen traza unidad
Estas dos características de hecho definen un proyector.

In [ ]:
P=projector(random_state(4));
@show norm(P*P-P);
@show trace(P);

In [ ]:
sigmas;
psi=random_state(2);

In [ ]:
# Nótese que el valor esperado es real. De que es consecia esto? 
trace(projector(psi)*sigma_x)

In [ ]:
"""
Function that takes a pure 2-level state and returns the Bloch sphere representation
"""
function stateToBloch(psi::Array)
    lista = Float64[]
    for sigma in sigmas
        push!(lista, real(trace(sigma*projector(psi))))
    end
    lista
end

In [ ]:
stateToBloch(psi)

Podemos notar que los eigenestados de las matrices de Pauli estan sobre cada uno de los ejes. Por ejemplo, los estados propios de $\sigma_z$ estan en $\pm \hat k$

In [ ]:
@show stateToBloch([1., 0.]);
@show stateToBloch([0., 1.]);

In [ ]:
for NumeroMatriz in eachindex(sigmas)
    sigma=sigmas[NumeroMatriz]
    λs, vs=eig(sigma)
    for i in eachindex(λs)
        λ=λs[i]
        v=vs[:,i]
        print("Numbero de matriz $(NumeroMatriz), eigenvalor $(λ)\n")
        print("eigenvector [$(v)],\n")
        print("eigenvector [$(stateToBloch(v))]\n\n")
    end
end

El concepto de matriz de densidad de densidad se usa cuando tenemos una superosición estadístisca de varios estados. En este caso, tenemos cierta probabilidad $p_i$ de tener un estado $|\psi_i\rangle$, entonces la matriz de densidad correspondiente es
$$
\rho = \sum_i p_i |\psi_i\rangle\langle\psi_i|.
$$
La matriz de densidad permite introducir el concepto de ensamble en mecánica cuantica. Tambien permite introducir el concepto de ignorancia, por ejemplo con respecto a una medición. Dos observadores, uno con mas información que otro, van a describir el mismo estado con diferentes objetos matemáticos. Esto nos hace reflexionar acerca de la validez o "realidad" de la función de onda (o de la matriz de densidad). Una cita de Peres es bastante util en este punto:

_Many physicists, perhaps a majority, have an intuitive, realistic worldview and consider a quantum state as a physical entity. Its value may not be known, but in principle the quantum state of a physical system would be well defined. However, there is no experimental evidence whatsoever to support this naive belief. On the contrary, if this view is taken seriously, it may lead to bizarreconsequences,called "quantumparadoxes."_

Los valores esperados se calculan como se explicó arriba, y si antes haciamos la evolución como 
$$
|\psi(t)\rangle= U(t)|\psi(0)\rangle
$$
ahora simplemente la hacemos como 
$$
\rho=U(t)\psi(0)U^\dagger(t)
$$
Los postulados de medición funcionan de manera similar, es decir, en vez de proyectar 
$$
|\psi\rangle \to \propto P_m |\psi\rangle
$$
proyectamos a 
$$
\rho \to \propto P_m\rho P_m.
$$


# Estados aleatorios

Decir que uno tiene un conjunto aleatorio tiene sentido si, ademas de decir cual es ese conjunto, dice uno cual es la medida. Uno puede tener numeros aleatorios de 0 a 1, pero los puede escoger de manera uniforme, o de cualquier otra manera. Claramente ambos casos son diferentes. 

En el caso de estados cuánticos, vamos a escoger estados aleatorios, de tal manera que no depende de la base que usemos para escogerlos. Si tomamos en forma ingenua los estados, podemos ver como se ven en la esfera de Bloch.


In [ ]:
e1=stateToBloch(random_state())

In [ ]:
NumberOfPoints=2700;
e4=zeros(NumberOfPoints,3);
euniform=zeros(NumberOfPoints,3);

In [ ]:
for i=1:NumberOfPoints
    e4[i,:]=stateToBloch(random_state())
end

In [ ]:
plot(e4[:,1],e4[:,2],e4[:,3],marker=(5,1,stroke(1)),w=0,legend=false,aspectratio=1)

In [ ]:
thetas=rand(NumberOfPoints)*π;
phis=rand(NumberOfPoints)*2*π;
PointsUniformDistribution=zeros(NumberOfPoints,3);

for i=1:NumberOfPoints
    euniform[i,:]=[sin(thetas[i])*cos(phis[i]) sin(thetas[i])*sin(phis[i]) cos(thetas[i])]
end

In [ ]:
plot(euniform[:,1],euniform[:,2],euniform[:,3],marker=(5,1,stroke(1)),w=0,legend=false,aspectratio=1)

## Tarea

* Pensar en una generalizacion de la esfera de Bloch para un qutrit (sistema de 3 niveles) y para un sistema de dos qubits. Programar una rutina que efectue dicho calculo. 

* Compare el calculo de la traza de un operador hermitico aleatorio exacto con el valor esperado con respecto a un solo estado aleatorio:
  * Construya un operador aleatorio hermítico, partiendo de un arreglo de $n\times n$ de números aleatorios complejos (sea esa matriz $A$). El observable a usar será $H=A+A^\dagger$ (muestre numérica que efectivamente es hermítico.
  * Use un estado aleatorio de la dimension correspondiente para calcular el valor esperado.
  * Compare con el valor de $\text{tr} H$. Debe ser muy cercano, cuando lo dividimos por la dimension del sistema. 
  * Calcule la diferencia como función de la dimensión del sistema y grafiquelo. 
  * Repita el ejercicio para una familia de operadores que varien con la dimensión pero que sean acotados (el espectro debe permanecer acotado a medida que la dimensión aumenta).



# Por hacer para la siguiente iteracion:
* Hacer demostracion numerica en 2 dimensiones que la forma de tomar direcciones aleatorias usando variables gaussianas y usando variables uniformes no da lo mismo. 
* Pensar si hacemos type_stable ahora, o quizá más adelante.
* Pensar si realmente vale la pena hacer que los arreglos sean de 1xn o mejor de nx1 para los estados aleatorios
* Hacer lentamente la diferencia entre vectores de 3*1 y de 1*3. Mostrar qeu al multiplicar por matrices tenemos problemas. Hacer que ellos piensen que es una maultiplicacion de un bra por un ket y de un ket por un bra.
* Revisar si la sugerencia de luis, aca, tiene efecto: la construcción `sigma_y=[0. -im; im 0]` y `sigma_y=[[0., im] [-im, 0]]` dan lo mismo, a veces vale la pena la segunda ya que el índice rápido son las columnas de una matriz.
* En [21], en la última línea tienes `sigmas=Array[sigma_x, sigma_y, sigma_z]`. Es mejor que le quites el Array, o que lo hagas más específico. Si haces `typeof(sigmas)` en [21] obtienes `Array{Array,1}`, lo que es un tipo abstracto (lo que se traduce en lento, más o menos equivalente a `Any`). Si, en cambio, haces `sigmas=[sigma_x, sigma_y, sigma_z]` entonces el tipo será Array{Array{Complex{Float64},2},1}. La sutileza es que todos los elementos de `sigmas` serán matrices complejas, aunque `sigma_x` seguirá siendo una matriz de Float64. (Algo parecido ocurren en una celda más abajo, donde defines \rho_A (que lindo es UTF!)




## Sugerencias de Luis:
* 